# ResNet

In [ ]:
import os
import numpy as np
import pickle
import keras
import tensorflow as tf
import warnings
from matplotlib import pyplot as plt
from IPython.display import clear_output

from keras.models import Sequential, Input, Model
from keras.layers import Dense, Dropout, Flatten, LSTM, Embedding, Activation, Add
from keras.layers import Conv2D, MaxPooling2D, AveragePooling2D
from keras.layers.normalization import BatchNormalization
from keras.layers.advanced_activations import LeakyReLU, ReLU, ELU
from keras.regularizers import l2 # L2-regularisation
from keras.models import Model
from keras import optimizers
from keras.layers.advanced_activations import LeakyReLU

warnings.filterwarnings("ignore")

### Data loader

In [ ]:
def load_data(data_name):
    pkl_file = open(data_name + '.pkl', 'rb')
    data = pickle.load(pkl_file)
    pkl_file.close()
    return data


print('Loading data...')
train = np.array(load_data('../input/neuraldataset2540/neural_train'))
train_label = np.array(load_data('../input/neuraldataset2540/neural_train_label'))
test = np.array(load_data('../input/neuraldataset2540/neural_test'))
test_label = np.array(load_data('../input/neuraldataset2540/neural_test_label'))
valid = np.array(load_data('../input/neuraldataset2540/neural_valid'))
valid_label = np.array(load_data('../input/neuraldataset2540/neural_valid_label'))
print('Data are loaded')

## ResNet

Layer blocks

In [ ]:
def relu_bn(inputs):
    relu = (ELU(alpha=0.1))(inputs)
    
    return relu


def residual_block_identity(x, filters, l2_lambda):
    
    f1, f2 = filters
    
    y = (Conv2D(filters = f1, 
                kernel_size = (5, 5),
                activation = 'linear', 
                W_regularizer = l2(l2_lambda), 
                padding = 'same'))(x)
    
    y = (LeakyReLU(alpha=0.1))(y)
    
    y = (Conv2D(filters = f2, 
                kernel_size = (5, 5), 
                activation = 'linear', 
                W_regularizer = l2(l2_lambda), 
                padding = 'same'))(y)
    
    y = (LeakyReLU(alpha=0.1))(y)
    
    out = (Add())([x, y])
    
    out = (MaxPooling2D(pool_size = (2, 2),
                        padding = 'same'))(out)
    
    out = (Dropout(0.3))(y)
    
    return out


def residual_block_convolutional(x, l2_lambda):
            
    y = (Conv2D(filters = 16, 
               kernel_size = (1, 1),
               strides = (2, 2),
               activation = 'linear'))(x)
    y = relu_bn(y)
    
    y = (LeakyReLU(alpha=0.1))(y)
    
    y = (Conv2D(filters = 32, 
               kernel_size = (5, 5),
               padding = "same", 
               activation ='linear'))(y)
    y = relu_bn(y)

    y = (LeakyReLU(alpha=0.1))(y)
    
    x = (Conv2D(filters = 32, 
               kernel_size = (1, 1),
               strides = (2, 2),
               padding = "same", 
               activation ='linear'))(x)
    
    y = (LeakyReLU(alpha=0.1))(y)
        
    out = (Add())([x, y])
    out = (MaxPooling2D(pool_size = (2, 2), 
                        padding = 'same'))(out)
    out = (Dropout(0.1))(out)
    
    return out

ResNet

In [ ]:
def resnet(out_shape, mm, input_shape = (64, 64, 3), classes = 2):
    
    l2_lambda = 0.0001
    num_filters = np.array([16, 32])
    num_blocks_list = [2, 4]
    
    inputs = Input(input_shape)
    
    t = (BatchNormalization())(inputs)
    t = (Conv2D(kernel_size = (mm, 5),
               strides = (1, 1),
               filters = 32,
               padding = "same"))(inputs)
        
    for i in range(len(num_blocks_list)):
        
        num_blocks = num_blocks_list[i]
        t = residual_block_convolutional(inputs, l2_lambda = l2_lambda)
        
        for j in range(num_blocks):
            
            t = residual_block_identity(t, filters = num_filters, l2_lambda = l2_lambda)
    
    layer4 = (Flatten())(t)
    layer4 = (BatchNormalization())(layer4)
    layer4 = (Dense((5 * mm), 
                    activation = 'linear', 
                    W_regularizer = l2(l2_lambda)))(layer4)
    layer4 = (LeakyReLU(alpha = 0.1))(layer4)
    layer4 = (Dropout(0.1))(layer4)
    finish = []
    
    for i in range(int((mm ** 2 - mm) / 2)):
        output = Dense(out_shape, activation = 'softmax', name = 'main_output'+str(i))(layer4)
        finish.append(output)

    model = Model(inputs = [inputs], outputs = finish)
    model.compile(loss = "categorical_crossentropy", 
                  optimizer=keras.optimizers.Adam(lr = 0.001))

    return model

Training

In [ ]:
batch_size = 8
epochs = 25
mm = 40

model = resnet(np.shape(train_label)[2], mm, input_shape = (train.shape[1], train.shape[2], 1))

history = model.fit(train, list(train_label), batch_size = batch_size, epochs = epochs, verbose = 1,
                          validation_data=(np.array(valid), list(valid_label)))

Making predictions

In [ ]:
predicted = model.predict(test)

print(np.shape(predicted))
print(np.shape(test_label))

output = open('predicted' + '.pkl', 'wb')
pickle.dump(predicted, output)
output.close()

Plot the loss

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()